In [2]:
import pandas as pd

## predict par model probas

### avec f1

In [3]:
# predict_ivan = pd.read_csv('predict_no_label/best_predict_logit_no_label/convnext_logits.csv')
predict_matis = pd.read_csv('final_predict/efficientnet_label_logit.csv')
predict_axel = pd.read_csv('final_predict/VIT_MIN4_NEWLABEL_CNNVIT_softmax.csv')
predict_mehdi = pd.read_csv('final_predict/predictions_v6_kaggle.csv')
predict_ivan = pd.read_csv('final_predict/predictions_logits_convnext_new_labels.csv')

In [4]:
predict_matis['gt'] = predict_matis['gt'].apply(eval)
columns = [f'logit_class_{i}' for i in range(9)]
matis_expanded = pd.DataFrame(predict_matis['gt'].tolist(), columns=columns)
predict_matis = pd.concat([predict_matis, matis_expanded], axis=1)
predict_matis = predict_matis.drop(columns=['gt'])


predict_axel['gt'] = predict_axel['gt'].apply(eval)
columns = [f'logit_class_{i}' for i in range(9)]
axel_expanded = pd.DataFrame(predict_axel['gt'].tolist(), columns=columns)
predict_axel = pd.concat([predict_axel, axel_expanded], axis=1)
predict_axel = predict_axel.drop(columns=['gt'])


In [7]:
f1_scores = {
    "axel": {
        '0': 0.7797,
        '1': 0.8663,
        '2': 0.7158,
        '3': 0.8706,
        '4': 0.4615,
        '5': 0.8235,
        '6': 0.8293,
        '7': 0.6813,
        '8': 0.9635
    },   
    "mehdi": {
        '0': 0.78,
        '1': 0.86,
        '2': 0.74,
        '3': 0.88,
        '4': 0.64,
        '5': 0.75,
        '6': 0.93,
        '7': 0.67,
        '8': 0.98
    },
    'matis': {
        '0': 0.7046,
        '1': 0.8770,
        '2': 0.7742,
        '3': 0.8095,
        '4': 0.5676,
        '5': 0.7642,
        '6': 0.8936,
        '7': 0.6067,
        '8': 0.8951
    },
    'ivan': {
        '0': 0.84,
        '1': 0.90,
        '2': 0.81,
        '3': 0.88,
        '4': 0.69,
        '5': 0.91,
        '6': 0.91,
        '7': 0.81,
        '8': 1.00
    },
}       

In [8]:
predict_mehdi.head(2)

,idx,logit_class_0,logit_class_1,logit_class_2,logit_class_3,logit_class_4,logit_class_5,logit_class_6,logit_class_7,logit_class_8
0,0.94854685397039150.77102236879446530.26465590...,0.003108,0.003475,0.017626,0.002762,0.003891,0.003414,0.008675,0.005815,0.951236
1,0.94538489270321890.86416385036243080.41813474...,0.001735,0.000565,0.000699,0.000761,0.000647,0.992827,0.000857,0.001239,0.000670


In [9]:
for class_idx, factor in f1_scores.get('mehdi', {}).items():
    column_name = f"logit_class_{class_idx}"
    if column_name in predict_mehdi.columns:
        predict_mehdi[column_name] *= factor
        
for class_idx, factor in f1_scores.get('axel', {}).items():
    column_name = f"logit_class_{class_idx}"
    if column_name in predict_axel.columns:
        predict_axel[column_name] *= factor
        
for class_idx, factor in f1_scores.get('matis', {}).items():
    column_name = f"logit_class_{class_idx}"
    if column_name in predict_matis.columns:
        predict_matis[column_name] *= factor
        
for class_idx, factor in f1_scores.get('ivan', {}).items():
    column_name = f"logit_class_{class_idx}"
    if column_name in predict_ivan.columns:
        predict_ivan[column_name] *= factor

In [10]:
predict_mehdi.head(2)

,idx,logit_class_0,logit_class_1,logit_class_2,logit_class_3,logit_class_4,logit_class_5,logit_class_6,logit_class_7,logit_class_8
0,0.94854685397039150.77102236879446530.26465590...,0.002424,0.002988,0.013043,0.002430,0.002490,0.002561,0.008068,0.003896,0.932211
1,0.94538489270321890.86416385036243080.41813474...,0.001354,0.000486,0.000517,0.000669,0.000414,0.744620,0.000797,0.000830,0.000657


In [11]:
dfs = [predict_ivan, predict_matis, predict_mehdi, predict_axel]

df_logit_sum = dfs[0].copy()
for df in dfs[1:]:
    df_logit_sum = df_logit_sum.merge(df, on=["idx"], suffixes=("", "_dup"), how="inner")
    for col in df.columns:
        if col.startswith("logit_class_") and col in df_logit_sum.columns:
            df_logit_sum[col] = df_logit_sum[col] + df_logit_sum.pop(f"{col}_dup")

In [12]:
df_logit_sum

,idx,logit_class_0,logit_class_1,logit_class_2,logit_class_3,logit_class_4,logit_class_5,logit_class_6,logit_class_7,logit_class_8
0,0.210808379701964530.57213720244606410.6181084...,0.995539,0.002306,0.104312,0.003875,1.412383,0.012018,0.006596,0.133350,0.004549
1,0.67252060376544930.285748381441017550.0081774...,0.044444,3.424101,0.001736,0.010097,0.001240,0.003940,0.005491,0.000705,0.003635
2,0.281356338373528360.59751877101018620.8428496...,0.089704,0.016838,0.056218,0.070669,0.036345,0.015411,0.015774,0.016456,3.442078
3,0.196301919483073630.95635162704319810.2744510...,0.181293,0.445525,1.986767,0.004145,0.006008,0.017848,0.019709,0.428803,0.005494
4,0.3659373525175630.40058162101475660.570281643...,0.869168,0.005473,0.677699,0.007786,0.587416,0.006352,0.009710,0.695252,0.014662
...,...,...,...,...,...,...,...,...,...,...
1339,0.41562174621411510.464442157775335040.3401925...,0.273814,0.671161,0.758193,0.056489,0.064826,0.263865,0.089196,0.041824,1.225349
1340,0.51889779340420590.28404831026445110.07885990...,1.987821,0.091044,0.912611,0.025967,0.023391,0.015242,0.019855,0.029977,0.005249
1341,0.64190728930123280.302094112954622630.2448779...,0.026708,0.002123,0.001698,0.001946,0.159842,0.011413,0.001485,0.005510,3.545717
1342,0.94538489270321890.86416385036243080.41813474...,0.010588,0.000499,0.000522,0.000711,0.000417,3.232267,0.000812,0.000840,0.000715


In [13]:
df_logit_sum["gt"] = df_logit_sum.iloc[:, 1:].idxmax(axis=1).str.extract("(\d+)").astype(int)

In [14]:
df_logit_sum[['idx', 'gt']].to_csv('predict/majorite_label_f1_logit_bst.csv', index=False)

### sans f1

In [4]:
predict_ivan = pd.read_csv('./predict/predictions_ivan_logit.csv')
predict_matis = pd.read_csv('./predict/efficientnet_full_label_logit.csv')
predict_mehdi = pd.read_csv('./predict/predictions_logit_mehdi.csv')
predict_axel = pd.read_csv('./predict/vit_label_logits.csv')

In [ ]:
predict_axel['gt'] = predict_axel['gt'].apply(eval)
columns = [f'logit_class_{i}' for i in range(9)]
axel_expanded = pd.DataFrame(predict_axel['gt'].tolist(), columns=columns)
predict_axel = pd.concat([predict_axel, axel_expanded], axis=1)

predict_matis['gt'] = predict_matis['gt'].apply(eval)
columns = [f'logit_class_{i}' for i in range(9)]
matis_expanded = pd.DataFrame(predict_matis['gt'].tolist(), columns=columns)
predict_matis = pd.concat([predict_matis, matis_expanded], axis=1)

In [101]:
predict_matis = predict_matis[['idx', 'logit_class_0', 'logit_class_1', 'logit_class_2', 'logit_class_3', 'logit_class_4', 'logit_class_5', 'logit_class_6', 'logit_class_7', 'logit_class_8']]
predict_matis["predict_matis"] = predict_matis.iloc[:, 1:].idxmax(axis=1).str.extract("(\d+)").astype(int)
predict_matis = predict_matis[['idx','predict_matis']]

predict_axel = predict_axel[['idx', 'logit_class_0', 'logit_class_1', 'logit_class_2', 'logit_class_3', 'logit_class_4', 'logit_class_5', 'logit_class_6', 'logit_class_7', 'logit_class_8']]
predict_axel["predict_axel"] = predict_axel.iloc[:, 1:].idxmax(axis=1).str.extract("(\d+)").astype(int)
predict_axel = predict_axel[['idx','predict_axel']]

predict_mehdi = predict_mehdi[['idx', 'logit_class_0', 'logit_class_1', 'logit_class_2', 'logit_class_3', 'logit_class_4', 'logit_class_5', 'logit_class_6', 'logit_class_7', 'logit_class_8']]
predict_mehdi["predict_mehdi"] = predict_mehdi.iloc[:, 1:].idxmax(axis=1).str.extract("(\d+)").astype(int)
predict_mehdi = predict_mehdi[['idx','predict_mehdi']]

predict_ivan = predict_ivan[['idx', 'logit_class_0', 'logit_class_1', 'logit_class_2', 'logit_class_3', 'logit_class_4', 'logit_class_5', 'logit_class_6', 'logit_class_7', 'logit_class_8']]
predict_ivan["predict_ivan"] = predict_ivan.iloc[:, 1:].idxmax(axis=1).str.extract("(\d+)").astype(int)
predict_ivan = predict_ivan[['idx','predict_ivan']]

In [102]:
def trouver_majorite(df, colonnes):
    def majorite_ligne(row):
        valeurs = [row[colonne] for colonne in colonnes]
        
        valeur_counts = {v: valeurs.count(v) for v in set(valeurs)}
        
        if len(valeur_counts) == 1:
            return list(valeur_counts.keys())[0]

        max_occurrences = max(valeur_counts.values())
        majoritaires = [v for v, count in valeur_counts.items() if count == max_occurrences]
        
        if len(majoritaires) == 1:
            return majoritaires[0]
        
        return row['predict_axel']

    df['gt'] = df.apply(majorite_ligne, axis=1)
    return df

In [ ]:
predict = pd.merge(predict_ivan, predict_axel, on=['idx'], how='left')
predict = pd.merge(predict, predict_mehdi, on=['idx'], how='left')
predict = pd.merge(predict, predict_matis, on=['idx'], how='left')
predict

,idx,predict_ivan,predict_axel,predict_mehdi
0,0.210808379701964530.57213720244606410.6181084...,0,0,0
1,0.67252060376544930.285748381441017550.0081774...,1,1,1
2,0.281356338373528360.59751877101018620.8428496...,8,8,3
3,0.196301919483073630.95635162704319810.2744510...,0,1,1
4,0.3659373525175630.40058162101475660.570281643...,0,7,7
...,...,...,...,...
1339,0.41562174621411510.464442157775335040.3401925...,0,8,0
1340,0.51889779340420590.28404831026445110.07885990...,0,2,0
1341,0.64190728930123280.302094112954622630.2448779...,8,8,8
1342,0.94538489270321890.86416385036243080.41813474...,5,5,5


In [ ]:
colonnes = ['predict_axel', 'predict_ivan','predict_mehdi','predict_matis']
final_predict = trouver_majorite(predict, colonnes)
final_predict

,idx,predict_ivan,predict_axel,predict_mehdi,gt
0,0.210808379701964530.57213720244606410.6181084...,0,0,0,0
1,0.67252060376544930.285748381441017550.0081774...,1,1,1,1
2,0.281356338373528360.59751877101018620.8428496...,8,8,3,8
3,0.196301919483073630.95635162704319810.2744510...,0,1,1,1
4,0.3659373525175630.40058162101475660.570281643...,0,7,7,7
...,...,...,...,...,...
1339,0.41562174621411510.464442157775335040.3401925...,0,8,0,0
1340,0.51889779340420590.28404831026445110.07885990...,0,2,0,0
1341,0.64190728930123280.302094112954622630.2448779...,8,8,8,8
1342,0.94538489270321890.86416385036243080.41813474...,5,5,5,5


In [105]:
final_predict[['idx','gt']].to_csv('./predict/predictions_models_sans_matis.csv', index=False)

## predict probas

In [36]:
predict_ivan = pd.read_csv('./predict/predictions_ivan_logit.csv')
predict_matis = pd.read_csv('./predict/efficientnet_full_label_logit.csv')
predict_mehdi = pd.read_csv('./predict/predictions_logit_mehdi.csv')
predict_axel = pd.read_csv('./predict/vit_label_logits.csv')

predict_axel['gt'] = predict_axel['gt'].apply(eval)
columns = [f'logit_class_{i}' for i in range(9)]
axel_expanded = pd.DataFrame(predict_axel['gt'].tolist(), columns=columns)
predict_axel = pd.concat([predict_axel, axel_expanded], axis=1)

predict_matis['gt'] = predict_matis['gt'].apply(eval)
columns = [f'logit_class_{i}' for i in range(9)]
matis_expanded = pd.DataFrame(predict_matis['gt'].tolist(), columns=columns)
predict_matis = pd.concat([predict_matis, matis_expanded], axis=1)

In [37]:
predict_mehdi.head(4)

,idx,logit_class_0,logit_class_1,logit_class_2,logit_class_3,logit_class_4,logit_class_5,logit_class_6,logit_class_7,logit_class_8
0,0.94854685397039150.77102236879446530.26465590...,0.485247,0.009827,0.004422,0.319037,0.013611,0.103216,0.022005,0.005934,0.036701
1,0.94538489270321890.86416385036243080.41813474...,0.002186,0.000213,0.000137,0.000754,0.000189,0.995870,0.000211,0.000138,0.000302
2,0.14366869116263460.86148507683249480.31931639...,0.021741,0.167069,0.006653,0.743122,0.003490,0.027683,0.007342,0.004561,0.018339
3,0.3659373525175630.40058162101475660.570281643...,0.292884,0.003889,0.037149,0.025882,0.090549,0.012392,0.007647,0.495550,0.034058


In [38]:
dfs = [predict_ivan, predict_matis, predict_mehdi, predict_axel]

df_logit_sum = dfs[0].copy()
for df in dfs[1:]:
    df_logit_sum = df_logit_sum.merge(df, on=["idx"], suffixes=("", "_dup"), how="inner")
    for col in df.columns:
        if col.startswith("logit_class_") and col in df_logit_sum.columns:
            df_logit_sum[col] = df_logit_sum[col] + df_logit_sum.pop(f"{col}_dup")

In [39]:
df_logit_sum = df_logit_sum.drop(columns=["gt","gt_dup"], axis=1)
df_logit_sum

,idx,logit_class_0,logit_class_1,logit_class_2,logit_class_3,logit_class_4,logit_class_5,logit_class_6,logit_class_7,logit_class_8
0,0.210808379701964530.57213720244606410.6181084...,2.185423,0.002233,0.123264,0.006522,0.760464,0.007786,0.006452,0.871758,0.036099
1,0.67252060376544930.285748381441017550.0081774...,0.579784,3.288071,0.013987,0.060777,0.003140,0.005244,0.025689,0.004217,0.019091
2,0.281356338373528360.59751877101018620.8428496...,0.311533,0.018455,0.013768,0.535320,0.025040,0.020764,0.016442,0.013167,3.045511
3,0.196301919483073630.95635162704319810.2744510...,0.936617,1.733699,1.095035,0.013559,0.006595,0.012100,0.004202,0.191438,0.006757
4,0.3659373525175630.40058162101475660.570281643...,1.411790,0.004159,0.643929,0.025963,0.733023,0.015116,0.008553,1.122005,0.035463
...,...,...,...,...,...,...,...,...,...,...
1339,0.41562174621411510.464442157775335040.3401925...,0.955023,0.832673,0.051401,0.208563,0.065550,0.416896,0.031477,0.094336,1.344081
1340,0.51889779340420590.28404831026445110.07885990...,1.839756,0.999407,1.087838,0.011266,0.001256,0.045313,0.002879,0.011530,0.000755
1341,0.64190728930123280.302094112954622630.2448779...,0.084304,0.001015,0.002828,0.003951,0.006857,0.003494,0.002154,0.028229,3.867167
1342,0.94538489270321890.86416385036243080.41813474...,0.029777,0.000221,0.000267,0.000777,0.000208,3.967970,0.000223,0.000238,0.000321


In [40]:
df_logit_sum["gt"] = df_logit_sum.iloc[:, 1:].idxmax(axis=1).str.extract("(\d+)").astype(int)

In [42]:
df_logit_sum[['idx','gt']].to_csv('./predict/majorite_logit.csv', index=False)

## Predict

In [66]:
predict_matis = pd.read_csv('./final_predict/efficientnet_label_predict.csv')
predict_axel = pd.read_csv('./final_predict/VIT_MIN4_NEWLABEL_CNNVIT.csv')
predict_ivan = pd.read_csv('./final_predict/predictions_convnext_new_labels.csv')
predict_mehdi = pd.read_csv('./final_predict/predictions_xgboostP2 (1).csv')

In [71]:
predict = pd.merge(predict_ivan, predict_axel, on=['idx'], how='left')
predict = pd.merge(predict, predict_matis, on=['idx'], how='left')
predict = predict.rename(columns={'gt_x': 'ivan', 'gt_y': 'axel', 'gt': 'matis'})
predict = pd.merge(predict, predict_mehdi, on=['idx'], how='left')
predict = predict.rename(columns={'gt': 'mehdi'})

In [72]:
predict

,idx,ivan,axel,matis,mehdi
0,0.210808379701964530.57213720244606410.6181084...,4,4,4,0
1,0.67252060376544930.285748381441017550.0081774...,1,1,1,1
2,0.281356338373528360.59751877101018620.8428496...,8,8,8,8
3,0.196301919483073630.95635162704319810.2744510...,2,7,2,2
4,0.3659373525175630.40058162101475660.570281643...,2,0,2,7
...,...,...,...,...,...
1339,0.41562174621411510.464442157775335040.3401925...,2,8,1,0
1340,0.51889779340420590.28404831026445110.07885990...,0,0,2,2
1341,0.64190728930123280.302094112954622630.2448779...,8,8,8,8
1342,0.94538489270321890.86416385036243080.41813474...,5,5,5,5


### F1 score le plus fort

In [98]:
f1_scores = {
    "axel": {
        '0': 0.7665,
        '1': 0.8449,
        '2': 0.6923,
        '3': 0.8916,
        '4': 0.5667,
        '5': 0.8393,
        '6': 0.8506,
        '7': 0.6136,
        '8': 0.9640
    },
    'mehdi': {
        '0': 0.78,
        '1': 0.89,
        '2': 0.71,
        '3': 0.92,
        '4': 0.57,
        '5': 0.76,
        '6': 0.94,
        '7': 0.70,
        '8': 0.93
    },
    'matis': {
        '0': 0.7459,
        '1': 0.8681,
        '2': 0.7800,
        '3': 0.7561,
        '4': 0.6769,
        '5': 0.7967,
        '6': 0.9195,
        '7': 0.6296,
        '8': 0.9241
    },
    'ivan': {
        '0': 0.76,
        '1': 0.93,
        '2': 0.77,
        '3': 0.90,
        '4': 0.69,
        '5': 0.83,
        '6': 0.93,
        '7': 0.72,
        '8': 1.00
    },
}         

In [99]:
def choose_best_f1_value(row, f1_scores):
    best_value = None
    best_f1_score = -float('inf') 

    for value in row:
        f1_scores_for_value = [f1_scores[model].get(str(value), 0) for model in f1_scores]
        
        # Trouver le modèle avec le meilleur F1 score pour cette classe
        max_f1_score = max(f1_scores_for_value)
        
        # Si ce F1 score est meilleur que le précédent, on met à jour la valeur
        if max_f1_score > best_f1_score:
            best_f1_score = max_f1_score
            best_value = value
    
    return best_value

In [101]:
predict['gt'] = predict.apply(lambda row: choose_best_f1_value(row, f1_scores), axis=1)

In [103]:
predict[['idx','gt']].to_csv('./predict/majorite_no_label_f1fort_nologit.csv', index=False)

### F1 score

In [102]:
predict

,idx,ivan,axel,matis,mehdi,gt
0,0.210808379701964530.57213720244606410.6181084...,0,0,4,4,0
1,0.67252060376544930.285748381441017550.0081774...,1,1,1,1,1
2,0.281356338373528360.59751877101018620.8428496...,8,8,8,8,8
3,0.196301919483073630.95635162704319810.2744510...,7,7,0,2,0
4,0.3659373525175630.40058162101475660.570281643...,7,4,2,7,2
...,...,...,...,...,...,...
1339,0.41562174621411510.464442157775335040.3401925...,3,8,5,8,8
1340,0.51889779340420590.28404831026445110.07885990...,0,2,0,0,0
1341,0.64190728930123280.302094112954622630.2448779...,8,8,8,8,8
1342,0.94538489270321890.86416385036243080.41813474...,5,5,5,5,5


In [90]:
def calculate_f1_sum(row):
    # Initialiser un dictionnaire pour stocker la somme des F1 scores par classe
    class_sums = {}
    
    # Parcours chaque colonne du DataFrame (ignoré 'idx' si elle existait)
    for col in row.index:
        if col != 'idx':  # Ignore la colonne 'idx' si elle existe
            class_idx = str(row[col])  # La classe est l'indice dans le dictionnaire f1_scores
            f1_score = f1_scores[col].get(class_idx, 0)  # Récupérer la valeur du F1 score
            
            if class_idx in class_sums:
                class_sums[class_idx] += f1_score
            else:
                class_sums[class_idx] = f1_score
    
    max_class = max(class_sums, key=class_sums.get)

    return class_sums

In [91]:
predict.apply(calculate_f1_sum, axis=1)

0                              {'0': 1.5265, '4': 1.2469}
1                               {'1': 3.5330000000000004}
2                               {'8': 3.8181000000000003}
3       {'7': 1.3336000000000001, '0': 0.7459, '2': 0.71}
4                     {'7': 1.42, '4': 0.5667, '2': 0.78}
                              ...                        
1339     {'3': 0.9, '8': 1.8940000000000001, '5': 0.7967}
1340                           {'0': 2.2859, '2': 0.6923}
1341                            {'8': 3.8181000000000003}
1342                                         {'5': 3.226}
1343               {'5': 2.3867000000000003, '0': 0.7665}
Length: 1344, dtype: object

In [ ]:
predict['gt'] = predict.apply(calculate_f1_sum, axis=1)

In [75]:
final_predict[['idx', 'majorite']].rename(columns={'majorite': 'gt'}).to_csv('./predict/majorite_no_label_nof1_nologit.csv', index=False)

In [58]:
predict[['idx', 'gt']].to_csv('./predict/majorite_no_label_f1_nologit.csv', index=False)

### Normal

In [73]:
def trouver_majorite(df, colonnes):
    def majorite_ligne(row):
        valeurs = [row[colonne] for colonne in colonnes]
        
        valeur_counts = {v: valeurs.count(v) for v in set(valeurs)}
        
        if len(valeur_counts) == 1:
            return list(valeur_counts.keys())[0]

        max_occurrences = max(valeur_counts.values())
        majoritaires = [v for v, count in valeur_counts.items() if count == max_occurrences]
        
        if len(majoritaires) == 1:
            return majoritaires[0]
        
        return row['ivan']

    df['majorite'] = df.apply(majorite_ligne, axis=1)
    return df

In [74]:
colonnes = ['ivan', 'axel', 'matis','mehdi']

final_predict = trouver_majorite(predict, colonnes)

In [77]:
final_predict.head(2)

,idx,ivan,axel,matis,mehdi,majorite
0,0.210808379701964530.57213720244606410.6181084...,4,4,4,0,4
1,0.67252060376544930.285748381441017550.0081774...,1,1,1,1,1


In [78]:
final_predict[['idx', 'majorite']].rename(columns={'majorite': 'gt'}).to_csv('./predict/majorite_label_nof1_nologit.csv', index=False)

In [80]:
final_predict[final_predict['majorite'] == predict['gt']]


,idx,ivan,axel,matis,mehdi,gt,majorite


In [80]:
def compter_lignes_diff(df):
    # Vérifier si les valeurs dans les colonnes 'ivan', 'axel', 'matis' sont différentes
    conditions = df[['ivan', 'axel', 'matis','mehdi']].apply(lambda row: len(set(row)) == 4, axis=1)
    
    # Compter le nombre de lignes où les trois valeurs sont distinctes
    return conditions.sum()

# Appel de la fonction
compter_lignes_diff(final_predict)

27